In [ ]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 0. Intro
1. 计算得到GT1 GT2 GT3对于testing data的NLL值 or KL-D作为metric
2. 不包含InferNet

# 1. Preparations
## 1.1 global settings

In [2]:
from MLP.utils import *
import os
from MLP.Config.config_base import BaseConfig
config = BaseConfig()
# For GT-2
from GT_model.GT_2.SA_for_PT_funcs_delta_eq1 import *
import GT_model.GT_2.SA_for_PT_funcs_delta_eq1
# reload(GT_model.GT_2.SA_for_PT_funcs_delta_eq1)
from importlib import reload
import metric_for_GTs_func

from metric_for_GTs_func import *
reload(metric_for_GTs_func)

<module 'metric_for_GTs_func' from 'D:\\Desktop\\PROJ\\PAProj\\data_handler\\cal_metric\\metric_for_GTs_func.py'>

In [ ]:
ARTIFICIAL = False
noise_pct = 0.05           # 噪音占比:我们希望生成的data总体上最多浮动的百分比noise_pct
# seed = 62            # [3,31,204,223,407]# 计算alpha和lambda的来源：NN的infer结果，还是来自csv的结果
q=1                 # q=1表示不使用quantile 计算metric
GT_w_Params = 2
## Real
seed_list = [3,31,204,223,407,62,508,626]

## Synth.
# seed_list = [4,31,204,35,407,66,508,512]


## 1.2 the data path


In [ ]:
# [4,31,204,35,407]
# Target data
if ARTIFICIAL:
        target_path = "../data/artificial_targets_v2_" + "noise=" + str(noise_pct)
else:
        target_path = r"../data/targets_all"

# Input: GT-2 params
if ARTIFICIAL:
        # GT-2 NN 产生的params
        params_opitim_path_NN = r"../data/SA_PT/params_artificial_GT2_noise=" + str(noise_pct)+"_seed=" + str(seed) + ".csv"
        # SA 的 params
        params_opitim_path_SA = "../data/SA_PT/params_opitim_artificial_v2_noise_"+str(noise_pct)+".csv"
else:
        # GT-2 NN 产生的params
        params_opitim_path_NN_2 = r"../data/SA_PT/params_GT2_seed=" + str(seed) + ".csv"
        params_opitim_path_NN_3 = r"../data/SA_PT/params_GT3_seed=" + str(seed) + ".csv"
        # SA 的 params
        params_opitim_path_SA = "../data/SA_PT/params_opitim_delta_T.csv"

# target data是target_5时 TARGET = 5
TARGET = 1

# Output path
# 3 col and 4 col
if ARTIFICIAL:
        NLL_metric_path_1  = r"../data/GT_metric/NLL_metric_GT_Tgt=1_e30_artificial_v_2_noise="+str(noise_pct)+".csv"
        NLL_metric_path_2  ="../data/GT_metric/NLL_metric_GT_Tgt=1_e30_all_"+"artificial_targets_v2_" + "noise=" + str(noise_pct)+"_seed="+str(seed)+".csv"
else:
        NLL_metric_path_1  ="../data/GT_metric/NLL_metric_GT_Tgt=1_e30.csv"
        # 添加的这一列和seed有关！
        NLL_metric_path_2  ="../data/GT_metric/NLL_metric_GT_Tgt=1_e30_seed="+str(seed)+".csv"

col_names = ['GT1', 'GT2(common)','GT2(SA)']
col_names_NN = ['GT2(NN)']

# For GT-2
alpha = -0.013581112
delta = 1
labda = 3.312402533

MIN_LOSS = 1e-30


In [ ]:
data_key = pd.read_csv(config.data_key_path,encoding="utf-8")

# params_opitim_delta= pd.read_csv(params_opitim_delta_path,encoding="utf-8")
params_opitim_SA = pd.read_csv(params_opitim_path_SA,encoding="utf-8")

# target_path里有全部的target data地址
target_all_path = os.listdir(target_path)
len_all = len(target_all_path)
print(f"一共*{len(data_key)}*场auction")

assert len_all == len(data_key),"wrong in len_all"


# 2. GT1全局只需要算一次
1. 生成一个file

In [ ]:
for i in idx:
        settings_df = data_key.iloc[idx,:]
        metric_pd.loc[pd_idx,col_names[0]] = GT1_KL(settings_df,target_df_uniq)


In [ ]:
GT1_metric_pd_NLL = pd.DataFrame()

for seed in seed_list:
        setup_seed(seed)
        shuffled_indices = save_data_idx_simplified(len_all)
        _,_,test_idx = get_data_idx(shuffled_indices, train_pct = config.train_pct, vali_pct=config.vali_pct)

        print(f"========== seed = {seed} ==========")

        if not ARTIFICIAL:
                params_opitim_path_NN_2 = r"../data/SA_PT/params_GT2_seed=" + str(seed) + ".csv"
                params_opitim_path_NN_3 = r"../data/SA_PT/params_GT3_seed=" + str(seed) + ".csv"
        else:
                params_opitim_path_NN_2 = r"../data/SA_PT/params_artificial_GT2_noise=" + str(noise_pct)+"_seed=" + str(seed) + ".csv"
                params_opitim_path_NN_3 = r"../data/SA_PT/params_artificial_GT3_noise=" + str(noise_pct)+"_seed=" + str(seed) + ".csv"

        params_opitim_NN_2 = pd.read_csv(params_opitim_path_NN_2,encoding="utf-8")
        params_opitim_NN_3 = pd.read_csv(params_opitim_path_NN_3,encoding="utf-8")

        pd_idx = 0
        metric_pd = pd.DataFrame()
        for i in test_idx:
                # Get target data
                target_path_i_path = os.path.join(target_path, target_all_path[i])
                target_df = pd.read_csv(target_path_i_path,encoding="utf-8")
                target = list(target_df.iloc[:,0])

                if target_df.shape[0] >= THRESHOLD:
                        pass
                else:   # 不计算少于THRESHOLD的auction
                        continue

                if not ARTIFICIAL:
                        target_df_uniq = pd.DataFrame(np.unique(target_df,axis=0)) # KL-D计算必须有这一步，去掉N值重复的sample
                else: # 对于生成数据，不包含P列，需要手动计算
                        target_df_group = target_df.groupby(by = ['N'],as_index=False).size()
                        target_df_group['P'] = target_df_group['size'] / sum(target_df_group['size'])

                        target_df_uniq = pd.merge(left = target_df, right=target_df_group ,how="left", on=['N'])
                        target_df_uniq.drop(labels='size',axis=1,inplace=True)
                        target_df_uniq = pd.DataFrame(np.unique(target_df_uniq,axis=0)).copy()    # KL-D计算必须有这一步，去掉N值重复的sample
                        target_df_uniq.columns = ['N','P']

                idx = 0

                # Get the auction setting
                settings_df = data_key.iloc[idx,:]

                alpha_2 = params_opitim_NN_2.loc[pd_idx,'alpha'].item()
                labda_2 = params_opitim_NN_2.loc[pd_idx,'labda'].item()
                alpha_3 = params_opitim_NN_3.loc[pd_idx,'alpha'].item()

                # metric_pd.loc[pd_idx,col_names[0]] = GT1_KL(settings_df,target_df_uniq)
                metric_pd.loc[pd_idx,col_names[0]] = GT2_KL(settings_df,target_df_uniq,alpha_2,labda_2)
                metric_pd.loc[pd_idx,col_names[1]] = GT3_KL(settings_df,target_df_uniq,alpha_3)

                pd_idx += 1

        # GT_metric_pd_KL.loc[seed,col_names[0]] = np.mean(metric_pd.loc[:,col_names[0]])
        GT_metric_pd_KL.loc[seed,col_names[0]] = np.mean(metric_pd.loc[:,col_names[0]])
        GT_metric_pd_KL.loc[seed,col_names[1]] = np.mean(metric_pd.loc[:,col_names[1]])


GT_metric_pd_KL.reset_index(inplace=True)
print(GT_metric_pd_KL)

In [ ]:
NLL_metric = np.array([0]*len(target_all_path)*3).reshape(-1,3)
NLL_metric_pd = pd.DataFrame(NLL_metric,columns=col_names)

for i in tqdm(range(len_all)):
        # Get target data
        target_path_i_path = os.path.join(target_path, target_all_path[i])
        target_df = pd.read_csv(target_path_i_path,encoding="utf-8")
        target = list(target_df.iloc[:,0])
        idx = 0
        # Get the auction setting
        if i < len_small:
                idx = i
                settings_df = data_key_small.iloc[idx,:]
        else:
                idx = i-len_small
                settings_df = data_key_large.iloc[idx,:]
        # Compute the NLL value of each GT models
        NLL_metric_pd.iloc[i,0] = GT_1(settings_df,target)
        # GT-2: common
        NLL_metric_pd.iloc[i,1] = GT_2(settings_df,target)

        # GT-2: SA
        alpha_i = params_opitim_SA.loc[i,'alpha'].item()
        labda_i = params_opitim_SA.loc[i,'labda'].item()
        NLL_metric_pd.iloc[i,2] = GT_2_uniq(settings_df,target,alpha_i,labda_i)